# Imports & Prepare Data

In [1]:
from models.ml_sequential import prepare_data, run_all
import tensorflow as tf
# Disable GPU training, comment out to enable
tf.config.set_visible_devices([], 'GPU')
# Imports used across models
from keras.layers import Input, Dense, TimeDistributed, Dropout
from keras.models import Sequential
from keras.regularizers import l2

features = ['female', 'age', 'height', 'mass', 'ta_set', 'rh_set']
features_scaler, output_scaler, X_padded, y_padded, max_len = prepare_data(features)

# Define the input/output shape
input_shape = (None, X_padded.shape[-1])
output_shape = y_padded.shape[-1]

# Train the model with early stopping
from keras.callbacks import EarlyStopping

# Custom log callback
class Callback(tf.keras.callbacks.Callback):
    SHOW_NUMBER = 10
    counter = 0
    epoch = 0

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch = epoch

    def on_epoch_end(self, epoch, logs=None):
        if self.counter == self.SHOW_NUMBER or self.epoch == 1:
            print(f'Epoch: {self.epoch}, loss: {logs["loss"]:.2f}', end='\r')
            # print(f'Epoch: {self.epoch}, loss: {logs["loss"]:.2f}, val loss: {logs["val_loss"]:.2f}', end='\r')
            if self.epoch > 1:
                self.counter = 0
        self.counter += 1

/Users/s5068337/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Max sequence length: 600


In [15]:
from tensorflow.keras.layers import GRU, BatchNormalization

# Sizes
model_sizes = [128]

# MODEL NAME
model_name = 'ml_gru_test'

for model_size in model_sizes:
    # Model architecture
    print("Running model size:", model_size)
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(GRU(model_size, return_sequences=True))
    model.add(GRU(64, return_sequences=True))
    model.add(GRU(32, return_sequences=True))
    model.add(TimeDistributed(Dense(output_shape)))

    # Compile the model
    model.compile(loss='mse', optimizer='adam')

    # Train the model
    # early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.fit(X_padded, y_padded, validation_split=0, epochs=300, batch_size=32, callbacks=[Callback()], verbose=0)
    print()

    # Run simulations using trained model
    run_all(model, model_name, features, features_scaler, output_scaler, max_len)
    print()

Running model size: 128
Epoch: 290, loss: 0.00
ml_gru_test
Average TRE RMSE: 0.4397916843410326
Average MTSK RMSE: 1.6221916434911903



In [7]:
from tensorflow.keras.layers import GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, TimeDistributed, Dense, Dropout
from tensorflow.keras.regularizers import l2
from kerastuner import HyperParameters, RandomSearch

model_name = 'test_gru'

def build_model(hp):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(GRU(units=hp.Int('gru_units', min_value=8, max_value=32, step=8), return_sequences=True))
    model.add(TimeDistributed(Dense(units=hp.Int('dense_units', min_value=4, max_value=16, step=4), activation='linear', kernel_regularizer=l2(0.01))))
    model.add(Dropout(rate=hp.Float('dropout_rate', min_value=0.1, max_value=0.3, step=0.1)))
    model.add(TimeDistributed(Dense(output_shape)))
    model.compile(loss='mse', optimizer='adam')
    return model

# Define the hyperparameter search space
hp = HyperParameters()
hp.Int('gru_units', min_value=8, max_value=32, step=8)
hp.Int('dense_units', min_value=4, max_value=16, step=4)
hp.Float('dropout_rate', min_value=0.1, max_value=0.3, step=0.1)

# Create the random search tuner
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    hyperparameters=hp,
    executions_per_trial=3,
    directory='tuner_dir',
    project_name='ml_gru_tuning'
)

# Perform the hyperparameter search
tuner.search(X_padded, y_padded, epochs=100, batch_size=32, validation_split=0.2)

# Get the best model and hyperparameters
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best hyperparameters: ", best_hyperparameters.values)

# Use the best model for further evaluation or predictions
best_model.fit(X_padded, y_padded, epochs=100, batch_size=32, verbose=0)

best_model.save('model_weights/{}.keras'.format(model_name))
# Run simulations using trained model
run_all(best_model, model_name, features, features_scaler, output_scaler, max_len)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
8                 |8                 |gru_units
8                 |8                 |dense_units
0.2               |0.2               |dropout_rate

Epoch 1/100
